In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import re
from sklearn.cluster import KMeans
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, PassiveAggressiveClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier,CatBoostRegressor
import os

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
test_data = pd.read_parquet('/kaggle/input/is-data/test_final.parquet')
train_data = pd.read_parquet('/kaggle/input/is-data/train_final.parquet')

In [3]:
def operator(text):
    text = text.lower()  
    
    if re.search(r'turkcell', text):
        return 'TURKCELL'
    elif re.search(r'vodafone', text):
        return 'VODAFONE'
    elif re.search(r'turk telekom|avea|turktelekom', text):
        return 'TURKTELEKOM'
    else:
        return 'OTHER'
    
def os(text):
    text = text.lower()  
    
    if re.search(r'huawei', text):
        return 'HarmanyOS'
    elif re.search(r'apple', text):
        return 'IOS'
    else:
        return 'ANDROID'
    

In [4]:
# menu_counts = {}

# for index, row in train_data.iterrows():
#     menus = row['target'].split(', ')
#     for menu in menus:
#         if row['id'] in menu_counts:
#             if menu in menu_counts[row['id']]:
#                 menu_counts[row['id']][menu] += 1
#             else:
#                 menu_counts[row['id']][menu] = 1
#         else:
#             menu_counts[row['id']] = {menu: 1}

# # Her kullanıcının toplamda en çok kullandığı menüleri bulma
# def find_most_used_menus(row):
#     if row['id'] not in menu_counts:
#         return None
#     max_count = max(menu_counts[row['id']].values())
#     if max_count == 1:
#         return None
#     most_used_menus = [menu for menu, count in menu_counts[row['id']].items() if count == max_count]
#     return ', '.join(most_used_menus)

# train_data['most_used_menus'] = train_data.apply(find_most_used_menus, axis=1)

# test_data['most_used_menus'] = test_data['id'].map(train_data.drop_duplicates(subset='id').set_index('id')['most_used_menus'])

In [5]:
# menu_dict = {}
# for index, row in train_data.iterrows():
#     id = row['id']
#     n_seconds_1 = row['n_seconds_1']
#     n_seconds_2 = row['n_seconds_2']
#     n_seconds_3 = row['n_seconds_3']
#     menus = row['target'].split(', ')
    
#     menu_dict[id] = {
#         menus[0]: n_seconds_1,
#         menus[1]: n_seconds_2,
#         menus[2]: n_seconds_3
#     }

# # Her 'id' için en çok vakit geçirilen menüyü bulma ve yeni sütuna ekleme
# def find_most_time_spent_menu(row):
#     id = row['id']
#     menu_times = menu_dict[id]
#     most_time_spent_menu = max(menu_times, key=menu_times.get)
#     return most_time_spent_menu

# train_data['most_time_spent_menu'] = train_data.apply(find_most_time_spent_menu, axis=1)

# test_data['most_time_spent_menu'] = test_data['id'].map(train_data.drop_duplicates(subset='id').set_index('id')['most_time_spent_menu'])

In [6]:
# train_data['most_used_menus'].fillna('', inplace=True)
# test_data['most_used_menus'].fillna('', inplace=True)

# train_data['most_time_spent_menu'].fillna('', inplace=True)
# test_data['most_time_spent_menu'].fillna('', inplace=True)

In [7]:
numeric_features = ['feature_' + str(i) for i in range(48,50)]
kmeans = KMeans(n_clusters=3)
train_data['cluster_labels'] = kmeans.fit_predict(train_data[numeric_features])

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [8]:
numeric_features = ['feature_' + str(i) for i in range(48,50)]
kmeans = KMeans(n_clusters=3)
test_data['cluster_labels'] = kmeans.fit_predict(test_data[numeric_features])

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [9]:
train_data['avg_seconds'] = (train_data['n_seconds_1'] + train_data['n_seconds_2'] + train_data['n_seconds_3']) / 3
test_data['avg_seconds'] = (test_data['n_seconds_1'] + test_data['n_seconds_2'] + test_data['n_seconds_3']) / 3

train_data['feature_std'] = train_data[numeric_features].std(axis=1)
test_data['feature_std'] = test_data[numeric_features].std(axis=1)

train_data['log_n_seconds_1'] = np.log(train_data['n_seconds_1'] + 1)
train_data['log_n_seconds_2'] = np.log(train_data['n_seconds_2'] + 1)
train_data['log_n_seconds_3'] = np.log(train_data['n_seconds_3'] + 1)

test_data['log_n_seconds_1'] = np.log(test_data['n_seconds_1'] + 1)
test_data['log_n_seconds_2'] = np.log(test_data['n_seconds_2'] + 1)
test_data['log_n_seconds_3'] = np.log(test_data['n_seconds_3'] + 1)

In [10]:
train_data['carrier'] = train_data['carrier'].apply(operator)
train_data['devicebrand'] =train_data['devicebrand'].apply(os)

test_data['carrier'] = test_data['carrier'].apply(operator)
test_data['devicebrand'] =test_data['devicebrand'].apply(os)

In [11]:
train_data["strata"] = train_data['devicebrand'] + train_data['carrier']
test_data["strata"] = test_data['devicebrand'] + test_data['carrier']

In [12]:
combined_data = pd.concat([train_data, test_data], axis=0)
encoder = OneHotEncoder()
combined_encoded = encoder.fit_transform(combined_data[["carrier", "devicebrand","strata"]])
train_encoded = combined_encoded[:len(train_data)]
test_encoded = combined_encoded[len(train_data):]

In [13]:
train_encoded = pd.DataFrame(train_encoded.toarray(), columns=encoder.get_feature_names_out(["carrier", "devicebrand","strata"]))
test_encoded = pd.DataFrame(test_encoded.toarray(), columns=encoder.get_feature_names_out(["carrier", "devicebrand","strata"]))
train_data = pd.concat([train_data, train_encoded], axis=1)
test_data = pd.concat([test_data, test_encoded], axis=1)
# train_data.drop(["carrier", "devicebrand"], axis=1, inplace=True)
# test_data.drop(["carrier", "devicebrand"], axis=1, inplace=True)

In [14]:
carrier_mean_encodings = train_data.groupby('carrier')[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].mean()
carrier_mean_encodings = carrier_mean_encodings.rename(columns={
    'n_seconds_1': 'carrier_n_seconds_1_mean',
    'n_seconds_2': 'carrier_n_seconds_2_mean',
    'n_seconds_3': 'carrier_n_seconds_3_mean'
})

# Calculate the mean of 'n_seconds_1', 'n_seconds_2', 'n_seconds_3' for each 'devicebrand'
devicebrand_mean_encodings = train_data.groupby('devicebrand')[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].mean()
devicebrand_mean_encodings = devicebrand_mean_encodings.rename(columns={
    'n_seconds_1': 'devicebrand_n_seconds_1_mean',
    'n_seconds_2': 'devicebrand_n_seconds_2_mean',
    'n_seconds_3': 'devicebrand_n_seconds_3_mean'
})

strata_mean_encodings = train_data.groupby('strata')[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].mean()
strata_mean_encodings = strata_mean_encodings.rename(columns={
    'n_seconds_1': 'strata_n_seconds_1_mean',
    'n_seconds_2': 'strata_n_seconds_2_mean',
    'n_seconds_3': 'strata_n_seconds_3_mean'
})


# Merge the mean encodings back into the original DataFrame
train_data = train_data.merge(carrier_mean_encodings, left_on='carrier', right_index=True, how='left')
train_data = train_data.merge(devicebrand_mean_encodings, left_on='devicebrand', right_index=True, how='left')
train_data = train_data.merge(strata_mean_encodings, left_on='strata', right_index=True, how='left')


In [15]:
carrier_mean_encodings = test_data.groupby('carrier')[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].mean()
carrier_mean_encodings = carrier_mean_encodings.rename(columns={
    'n_seconds_1': 'carrier_n_seconds_1_mean',
    'n_seconds_2': 'carrier_n_seconds_2_mean',
    'n_seconds_3': 'carrier_n_seconds_3_mean'
})

# Calculate the mean of 'n_seconds_1', 'n_seconds_2', 'n_seconds_3' for each 'devicebrand'
devicebrand_mean_encodings = test_data.groupby('devicebrand')[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].mean()
devicebrand_mean_encodings = devicebrand_mean_encodings.rename(columns={
    'n_seconds_1': 'devicebrand_n_seconds_1_mean',
    'n_seconds_2': 'devicebrand_n_seconds_2_mean',
    'n_seconds_3': 'devicebrand_n_seconds_3_mean'
})

strata_mean_encodings = test_data.groupby('strata')[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].mean()
strata_mean_encodings = strata_mean_encodings.rename(columns={
    'n_seconds_1': 'strata_n_seconds_1_mean',
    'n_seconds_2': 'strata_n_seconds_2_mean',
    'n_seconds_3': 'strata_n_seconds_3_mean'
})


# Merge the mean encodings back into the original DataFrame
test_data = test_data.merge(carrier_mean_encodings, left_on='carrier', right_index=True, how='left')
test_data = test_data.merge(devicebrand_mean_encodings, left_on='devicebrand', right_index=True, how='left')
test_data = test_data.merge(strata_mean_encodings, left_on='strata', right_index=True, how='left')

In [16]:
train_data['month'] = pd.to_datetime(train_data['month'], format='%m').dt.month
test_data['month'] = pd.to_datetime(test_data['month'], format='%m').dt.month

In [17]:
train_data['total_seconds'] = train_data['n_seconds_1'] + train_data['n_seconds_2'] + train_data['n_seconds_3']
test_data['total_seconds'] = test_data['n_seconds_1'] + test_data['n_seconds_2'] + test_data['n_seconds_3']

In [18]:
#Özelliklerin Ortalaması
train_data['mean_seconds'] = train_data[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].mean(axis=1)
test_data['mean_seconds'] = test_data[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].mean(axis=1)

In [19]:
#Özelliklerin Maksimumu
train_data['max_seconds'] = train_data[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].max(axis=1)
test_data['max_seconds'] = test_data[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].max(axis=1)

In [20]:
#Özelliklerin Minimumu
train_data['min_seconds'] = train_data[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].min(axis=1)
test_data['min_seconds'] = test_data[['n_seconds_1', 'n_seconds_2', 'n_seconds_3']].min(axis=1)

In [21]:
#Kategorik Değişkenlerin İstatistikleri
carrier_mean = train_data.groupby('carrier')['total_seconds'].mean().to_dict()
train_data['carrier_mean'] = train_data['carrier'].map(carrier_mean)
test_data['carrier_mean'] = test_data['carrier'].map(carrier_mean)

In [22]:
devicebrand_mean = train_data.groupby('devicebrand')['total_seconds'].mean().to_dict()
train_data['devicebrand_mean'] = train_data['devicebrand'].map(devicebrand_mean)
test_data['devicebrand_mean'] = test_data['devicebrand'].map(devicebrand_mean)

In [23]:
devicebrand_mean = train_data.groupby('strata')['total_seconds'].mean().to_dict()
train_data['strata_mean'] = train_data['strata'].map(devicebrand_mean)
test_data['strata_mean'] = test_data['strata'].map(devicebrand_mean)

In [24]:
train_data.drop(["carrier", "devicebrand",'strata'], axis=1, inplace=True)
test_data.drop(["carrier", "devicebrand","strata"], axis=1, inplace=True)

In [25]:
#Zamanla Değişen Özellikler
# train_data['total_seconds_diff'] = train_data.groupby('month')['total_seconds'].diff()
# test_data['total_seconds_diff'] = test_data.groupby('month')['total_seconds'].diff()

In [26]:
#Ay Bazlı İstatistikler********
monthly_stats_tr = train_data.groupby('month')['total_seconds'].agg(['mean', 'std', 'min', 'max']).reset_index()
monthly_stats_tr.columns = ['month', 'mean_total_seconds', 'std_total_seconds', 'min_total_seconds', 'max_total_seconds']
monthly_stats_te = test_data.groupby('month')['total_seconds'].agg(['mean', 'std', 'min', 'max']).reset_index()
monthly_stats_te.columns = ['month', 'mean_total_seconds', 'std_total_seconds', 'min_total_seconds', 'max_total_seconds']

train_data = train_data.merge(monthly_stats_tr, on='month', how='left')
test_data = test_data.merge(monthly_stats_te, on='month', how='left')

scaler = StandardScaler()

# train_data[["mean_total_seconds", "std_total_seconds", "min_total_seconds","max_total_seconds","total_seconds_diff"]] = scaler.fit_transform(train_data[["mean_total_seconds", "std_total_seconds", "min_total_seconds","max_total_seconds","total_seconds_diff"]])
# test_data[["mean_total_seconds", "std_total_seconds", "min_total_seconds","max_total_seconds","total_seconds_diff"]] = scaler.transform(test_data[["mean_total_seconds", "std_total_seconds", "min_total_seconds","max_total_seconds","total_seconds_diff"]])

train_data[["n_seconds_1", "n_seconds_2", "n_seconds_3"]] = scaler.fit_transform(train_data[["n_seconds_1", "n_seconds_2", "n_seconds_3"]])
test_data[["n_seconds_1", "n_seconds_2", "n_seconds_3"]] = scaler.transform(test_data[["n_seconds_1", "n_seconds_2", "n_seconds_3"]])

# numeric_features = ['feature_' + str(i) for i in range(1, 50)]
# train_data[numeric_features] = scaler.fit_transform(train_data[numeric_features])
# test_data[numeric_features] = scaler.transform(test_data[numeric_features])


In [27]:
model_cols = list(train_data.columns)
model_cols.remove('target')

model_cols.remove('id')

In [28]:
X = train_data[model_cols]
y = train_data["target"]

In [29]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, classification_report

catb_params ={ 
            'random_seed' :42 , 
            'iterations':1000,
            'early_stopping_rounds':20,
            'thread_count':10,
            }
model = CatBoostClassifier(**catb_params,task_type='GPU',devices='0:1',learning_rate=0.099762)
model.fit(X, y)
#y_pred = model.predict(X_val)
#accuracy = accuracy_score(y_val, y_pred)
#print("Accuracy:", accuracy)
#print("Classification Report:")
#print(classification_report(y_val, y_pred))

0:	learn: 4.1330038	total: 428ms	remaining: 7m 7s
1:	learn: 3.9622321	total: 604ms	remaining: 5m 1s
2:	learn: 3.8415616	total: 744ms	remaining: 4m 7s
3:	learn: 3.7503927	total: 878ms	remaining: 3m 38s
4:	learn: 3.6791842	total: 1.01s	remaining: 3m 21s
5:	learn: 3.6197016	total: 1.15s	remaining: 3m 10s
6:	learn: 3.5699951	total: 1.29s	remaining: 3m 2s
7:	learn: 3.5279027	total: 1.42s	remaining: 2m 55s
8:	learn: 3.4918021	total: 1.55s	remaining: 2m 50s
9:	learn: 3.4600740	total: 1.69s	remaining: 2m 47s
10:	learn: 3.4319331	total: 1.82s	remaining: 2m 43s
11:	learn: 3.4076485	total: 1.96s	remaining: 2m 41s
12:	learn: 3.3856886	total: 2.1s	remaining: 2m 39s
13:	learn: 3.3665157	total: 2.23s	remaining: 2m 37s
14:	learn: 3.3496920	total: 2.36s	remaining: 2m 35s
15:	learn: 3.3335263	total: 2.5s	remaining: 2m 33s
16:	learn: 3.3189199	total: 2.64s	remaining: 2m 32s
17:	learn: 3.3053614	total: 2.77s	remaining: 2m 31s
18:	learn: 3.2925450	total: 2.91s	remaining: 2m 30s
19:	learn: 3.2814703	total: 

In [30]:
test_predictions = model.predict(test_data[model_cols])

In [31]:
test_predictions

array([['menu6, menu7, menu2'],
       ['menu6, menu2, menu1'],
       ['menu2, menu8, menu4'],
       ...,
       ['menu6, menu2, menu4'],
       ['menu6, menu2, menu4'],
       ['menu6, menu2, menu4']], dtype=object)

In [32]:
tum_menuler = ['menu1', 'menu2', 'menu3', 'menu4', 'menu5', 'menu6', 'menu7', 'menu8', 'menu9']

def binary(tahmin, tum_menuler):
    binary_tahmin = ['0'] * len(tum_menuler)
    menu_listesi = tahmin.split(', ') if isinstance(tahmin, str) else tahmin  
    for menu_group in menu_listesi:
        for menu in menu_group.split(', '):
            if menu in tum_menuler:
                index = tum_menuler.index(menu)
                binary_tahmin[index] = '1'
    return ''.join(binary_tahmin)
binary_sonuclar = [binary(tahmin, tum_menuler) for tahmin in test_predictions]

In [33]:
binary_sonuclar

['010001100',
 '110001000',
 '010100010',
 '010011000',
 '010001010',
 '010101000',
 '010110000',
 '110001000',
 '010001010',
 '010001010',
 '110001000',
 '010110000',
 '010110000',
 '000111000',
 '010110000',
 '110001000',
 '010110000',
 '010101000',
 '110001000',
 '010101000',
 '110001000',
 '110001000',
 '010101000',
 '010110000',
 '110001000',
 '010100010',
 '010001010',
 '010101000',
 '110001000',
 '110001000',
 '010101000',
 '110001000',
 '010100010',
 '110001000',
 '010101000',
 '110001000',
 '010101000',
 '010101000',
 '010010010',
 '110001000',
 '110001000',
 '110001000',
 '110001000',
 '010100010',
 '110001000',
 '010011000',
 '010101000',
 '010100010',
 '110001000',
 '110001000',
 '010101000',
 '010101000',
 '110001000',
 '110001000',
 '110001000',
 '010101000',
 '010101000',
 '010110000',
 '010011000',
 '010001010',
 '010110000',
 '110001000',
 '010101000',
 '010011000',
 '010101000',
 '110001000',
 '010110000',
 '010001010',
 '110001000',
 '010110000',
 '110001000',
 '0101

In [34]:
subm = pd.read_parquet('/kaggle/input/is-data/submission_sample_final.parquet')

In [35]:
submission = pd.DataFrame({"id": test_data["id"], "target": binary_sonuclar})

In [36]:
submission.to_parquet("submission.parquet", index=False)
pd.read_parquet("submission.parquet")

,id,target
0,2e6105f5911256f4f6c4813ed,010001100
1,c56ad71dae0a5dbd3e7d36adc,110001000
2,4d02ea175f6581f0c6385311f,010100010
3,3412d27a86c286ba078fa935c,010011000
4,0203b561f6f7e10eafa46eefa,010001010
...,...,...
11950,7687113f46112edf4f56666ee,010100001
11951,5ff8eb7a06fd48b60dbc04f34,010101000
11952,ac23a7b9ad3e5d61e738c854b,010101000
11953,7da05018634ea2eee4b122756,010101000
